In [30]:
import wandb
wandb.login()

True

In [31]:
sweep_config = {
    'method': 'random',
    'name': 'sweep',
}

In [32]:
metric = {
    'name': 'epoch_loss',
    'goal': 'minimize',
}
sweep_config['metric'] = metric

In [34]:
parameters_dict= {
    'optimizer': {
        'values': ['AdamW', 'SGD']
    },
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
    },
        'batch_size': {
        # integers between 16 and 128
        # with evenly-distributed logarithms
        'distribution': 'q_log_uniform_values',
        'q': 4,
        'min': 16,
        'max': 128,
    },
    'latent_dim': {
        'values': [16,24,32]
    }
}
sweep_config["parameters"] = parameters_dict

In [41]:
sweep_id = wandb.sweep(sweep_config, project="bep-mia")

Create sweep with ID: qe8sdmcn
Sweep URL: https://wandb.ai/bepjustin-mia/bep-mia/sweeps/qe8sdmcn


In [42]:
from data import load_mrs
from config import wandb_config_VAE
from training import get_model, get_callback, get_pipeline, train
from models import MRS_encoder, MRS_decoder
from visualization import sample_model, show_generated_data
import wandb

In [43]:
def wandb_train(wandb_config=None):
    with wandb.init(config=wandb_config):
        train_dataset, eval_dataset, ppm = load_mrs()

        wandb_config = wandb.config
        training_config, model_config = wandb_config_VAE(wandb_config)

        wandb_callback = get_callback(training_config, model_config)
        encoder = MRS_encoder(model_config)
        decoder = MRS_decoder(model_config)

        model = get_model(model_config, encoder, decoder)

        pipeline = get_pipeline(model, training_config)

        trained_model = train('wandb_sweeps', pipeline, train_dataset, eval_dataset, wandb_callback)

In [46]:
wandb.agent(sweep_id, wandb_train, count=1)

wandb: Ctrl + C detected. Stopping sweep.
